In [2]:
import QuantLib as ql

In [12]:
# Playing around with basic date stuff
today = ql.Date(17,5,2023)

# Check if a date is a tuesday
today.weekday() == ql.Wednesday

# Check if a date is a tuesday
today.weekday() == ql.Sunday

# Arithmetic with dates
yesterday = today - 1
tomorrow = today + 1
print(yesterday)

a_year_from_now = today + ql.Period(1, ql.Years)

May 16th, 2023


In [19]:
# The Schedule object can be used to construct a list of dates such as coupon payments.
effectiveDate = ql.Date(15,6,2020)
terminationDate = ql.Date(15,6,2022)
frequency = ql.Period('6M')
calendar = ql.TARGET()
convention = ql.ModifiedFollowing
terminationDateConvention = ql.ModifiedFollowing
rule = ql.DateGeneration.Backward
endOfMonth = False
schedule = ql.Schedule(effectiveDate, terminationDate, frequency, calendar, convention, terminationDateConvention, rule, endOfMonth)
list(schedule)

[Date(15,6,2020),
 Date(15,12,2020),
 Date(15,6,2021),
 Date(15,12,2021),
 Date(15,6,2022)]

# Fixed rate bond pricing
Let's consider a hypothetical bond 
- par value of 100
- pays 6% coupon 
- semi-annually 
- issued on January 15th, 2015 
- mature on January 15th, 2016. 
The bond will pay a coupon on July 15th, 2015 and January 15th, 2016. 
The par amount of 100 will also be paid on the January 15th, 2016.

To make things simpler, let's assume that we know the spot rates of the treasury as of January 15th, 2015.
The annualized spot rates are 
- 0.5% for 6 months 
- 0.7% for 1 year point. 

## Example 1

In [43]:
# Evaluation of metrics will happen on this day
todays_date = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = todays_date

# Assuming a term structure for the calculation
spot_dates = [ql.Date(15, 1, 2015), ql.Date(15, 7, 2015), ql.Date(15, 1, 2016)]
spot_rates = [0.0, 0.005, 0.007]

# Day count convention
day_count = ql.Thirty360(ql.Thirty360.BondBasis)

# There are different types of calendars used in the US so explicitly have to be specified in the constructor
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)


interpolation = ql.Linear()
compounding = ql.Compounded
compounding_frequency = ql.Annual
spot_curve = ql.ZeroCurve(spot_dates, spot_rates, day_count, calendar, interpolation, compounding, compounding_frequency)
spot_curve_handle = ql.YieldTermStructureHandle(spot_curve)

issue_date = ql.Date(15, 1, 2015)
maturity_date = ql.Date(15, 1, 2016)
tenor = ql.Period(ql.Semiannual)


bussiness_convention = ql.Unadjusted
date_generation = ql.DateGeneration.Backward
month_end = False
schedule = ql.Schedule (issue_date, maturity_date, tenor, calendar, bussiness_convention, bussiness_convention , date_generation, month_end)
list(schedule)
#[Date(15, 1, 12015), Date(15,7,2015), Date(15,1,2016)]


# Now lets build the coupon
coupon_rate = .06
coupons = [coupon_rate]

# Now lets construct the FixedRateBond
settlement_days = 0
face_value = 100
fixed_rate_bond = ql.FixedRateBond(settlement_days, face_value, schedule, coupons, day_count)

# create a bond engine with the term structure as input;
# set the bond to use this bond engine
bond_engine = ql.DiscountingBondEngine(spot_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

# Finally the price 105.27653992490683
fixed_rate_bond.NPV()

105.27653992490683

## Example 2

In [ ]:
# We are valuing this bond at this date
valuation_date = ql.Date(30, 6, 2020)
ql.Settings.instance().evaluationDate = valuation_date

# Creating a schedule of cash flows
schedule = ql.Schedule(ql.Date(7, 5, 2016), ql.Date(15, 8, 2024), ql.Period(ql.Semiannual), ql.NullCalendar(), ql.Following, ql.Following, ql.DateGeneration.Forward, True)

fixedRateBond = ql.FixedRateBond(0, 100, schedule, [0.05], ql.ActualActual(ql.ActualActual.ISMA, schedule))

curve = ql.FlatForward(valuation_date, ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual360(), ql.Compounded)
handle = ql.YieldTermStructureHandle(curve)

bondEngine = ql.DiscountingBondEngine(handle)

fixedRateBond.setPricingEngine(bondEngine)

irr = fixedRateBond.bondYield(fixedRateBond.NPV(), ql.Actual360(), ql.Compounded, ql.Semiannual, valuation_date)
print('NPV:', fixedRateBond.NPV())
print('IRR:', irr)
print('Clean Price:', fixedRateBond.cleanPrice(irr, ql.Actual360(), ql.Compounded, ql.Semiannual, valuation_date))
print('Dirty Price:', fixedRateBond.dirtyPrice(irr, ql.Actual360(), ql.Compounded, ql.Semiannual, valuation_date))
print('Accrued Interest:', fixedRateBond.accruedAmount(ql.Date(30, 6, 2020)))
